In [1]:
from keras.datasets import imdb
import pandas as pd
import numpy as np
from keras.layers import LSTM, Activation, Dropout, Dense, Input
from keras.layers.embeddings import Embedding
from keras.models import Model
import string
import re
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split

In [2]:

data = pd.read_csv('/mnt/ufs18/home-053/hejaseba/IMDBDataset.csv')

data['review'] = data['review'].str.lower()

In [3]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]
             

In [4]:
def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')


In [6]:
reviews = data_without_stopwords['clean_review']

reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])
 
sentiment = data_without_stopwords['sentiment']

In [7]:
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))


In [8]:
X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)


In [9]:
tokenizer = Tokenizer(num_words=95419)
tokenizer.fit_on_texts(X_train)
words_to_index = tokenizer.word_index

In [10]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map
    

In [11]:
word_to_vec_map = read_glove_vector('/mnt/ufs18/home-053/hejaseba/glove.6B.300d.txt')


In [12]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len)) # (95419,50)

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

#embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)


In [13]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [14]:
import nltk, re, pprint, string
from nltk import word_tokenize, sent_tokenize
from nltk.util import ngrams
import random
import torch
from torch import nn


emb_ = torch.tensor(emb_matrix,dtype=torch.float64,device="cuda") # ,requires_grad=True


In [15]:
# experiments 2

X_train_2 = []
max_ = 1505 # then try 600 
for i in range(0,len(X_train)):
    seq_ = word_tokenize(X_train[i])
    X_train_2.append([])
    for k in range(0,len(seq_)):
        ind_ = 0
        if(seq_[k] in words_to_index.keys()):
            ind_ = words_to_index[seq_[k]]
            if(ind_>=95419):
                ind_ = 0
        X_train_2[i].append(ind_)
    
    if(len(X_train_2[i])<max_):
        
        for j in range(0,max_-len(X_train_2[i])):
            X_train_2[i].append(0)

In [16]:
X_test_2 = []
for i in range(0,len(X_test)):
    seq_ = word_tokenize(X_test[i])
    X_test_2.append([])
    for k in range(0,len(seq_)):
        ind_ = 0
        if(seq_[k] in words_to_index.keys()):
            ind_ = words_to_index[seq_[k]]
            if(ind_>=95419):
                ind_ = 0
        X_test_2[i].append(ind_)
    


In [56]:
y_ =torch.tensor(Y_test,device="cuda")
y_ = torch.reshape(y_,(1,10000)).float() # X

y_ = y_[0]


y =torch.tensor(Y_train,device="cuda")
y = torch.reshape(y,(1,40000)).float() # X
y = y[0]

Wf1 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Uf1 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
buf1 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 


Wi1 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Ui1 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
bui1 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 

Wg1 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Ug1 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
bug1 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 

Wo1 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Uo1 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
buo1 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 



Wf11 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Uf11 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
buf11 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 


Wi11 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Ui11 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
bui11 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 

Wg11 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Ug11 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
bug11 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 

Wo11 = torch.normal(0,0.01, size=(300,256),requires_grad=True,device="cuda") 
Uo11 = torch.normal(0,0.01, size=(256,256),requires_grad=True,device="cuda") 
buo11 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 


V2 = torch.normal(0,0.01, size=(512,256),requires_grad=True,device="cuda") 
bv2 = torch.zeros(size=(1,256),requires_grad=True,device="cuda") 

V = torch.normal(0,0.01, size=(256,2),requires_grad=True,device="cuda") 
bv = torch.zeros(size=(1,2),requires_grad=True,device="cuda")  



params = [Uf1,Wf1,buf1,Ug1,Wg1,bug1,Ui1,Wi1,bui1,Uo1,Wo1,buo1,V,bv,Uf11,Wf11,buf11,Ug11,Wg11,bug11,Ui11,Wi11,bui11,Uo11,Wo11,buo11,V2,bv2] #,emb_]



In [57]:

r_ = nn.ReLU().cuda()

tan_ = nn.Tanh().cuda()
m = nn.Sigmoid().cuda()

#m = nn.ReLU().cuda()

'''
def m(x):
    return (torch.sinh(x*0.2).cuda() + torch.asinh(x*0.2).cuda())

def tan_(x):
    return (torch.sinh(x*0.2).cuda() + torch.asinh(x*0.2))
'''
s_ = nn.Softmax(dim=1) # (1,2)

def models(X):     
        h1 = torch.zeros(size=(1,256),device="cuda")
        c1 = torch.zeros(size=(1,256),device="cuda")

        h11 = torch.zeros(size=(1,256),device="cuda")
        c11 = torch.zeros(size=(1,256),device="cuda")
        for i in range(0,len(X[0])): # 350 words
              ind = [X[s_][i] for s_ in range(0,1) ]                
              if(0 not in ind):
                    f1 = ( emb_[ind].float() @ Wf1 ) + (h1 @ Uf1) + buf1
                    f1 = m(f1)
                    k1 = c1 * f1
                    g1 = (h1 @ Ug1) + (emb_[ind].float() @ Wg1) + bug1
                    g1 = tan_(g1)
                    i1 = (h1 @ Ui1) + (emb_[ind].float() @ Wi1) + bui1
                    i1 = m(i1)
                    j1 = g1 * i1
                    c1 = j1 + k1 
                    o1 = (h1 @ Uo1) + (emb_[ind].float() @ Wo1) + buo1
                    o1 = m(o1)
                    h1 = o1 * tan_(c1)  
                    
              ind1 = [X[s_][len(X[0])-1-i] for s_ in range(0,1) ] 
                
              if(0 not in ind1):
                    f11 = ( emb_[ind1].float() @ Wf11 ) + (h11 @ Uf11) + buf11
                    f11 = m(f11)
                    k11 = c11 * f11
                    g11 = (h11 @ Ug11) + (emb_[ind1].float() @ Wg11) + bug11
                    g11 = tan_(g11)
                    i11 = (h11 @ Ui11) + (emb_[ind1].float() @ Wi11) + bui11
                    i11 = m(i11)
                    j11 = g11 * i11
                    c11 = j11 + k11 
                    o11 = (h11 @ Uo11) + (emb_[ind1].float() @ Wo11) + buo11
                    o11 = m(o11)
                    h11 = o11 * tan_(c11)
                    
        h3 = torch.cat((h1,h11),1)
        y2 = h3 @ V2 + bv2
        y2 = r_(y2)
        y = y2 @ V + bv
        
        y = s_(y)
        return y
    

def cross_entropy(y_hat,y):
    #yy = torch.tensor(y)
    to_ =  -torch.log(y_hat[range(len(y_hat)),y]+1e-8)
    #to_ = to_ / 50.0
    to_ =  to_.sum(0,keepdim=True)
    to_ = to_ / 1.0
    return to_
    
'''

def cross_entropy(y_hat,y):
    to_= -torch.log(y_hat[range(len(y_hat)),y])
    #to_ = to_ / 10.0
    return to_
    #return to_.sum(0,keepdim=True)
'''


'\n\ndef cross_entropy(y_hat,y):\n    to_= -torch.log(y_hat[range(len(y_hat)),y])\n    #to_ = to_ / 10.0\n    return to_\n    #return to_.sum(0,keepdim=True)\n'

In [58]:
def test(X,pass_=1):  
        h1 = torch.zeros(size=(1,256),device="cuda")
        c1 = torch.zeros(size=(1,256),device="cuda")

        h11 = torch.zeros(size=(1,256),device="cuda")
        c11 = torch.zeros(size=(1,256),device="cuda")
        if(pass_==11):
            len_=350
        else:
            len_ = len(X)
            
        
        for i in range(0,len_): # maybe take average
              ind = X[i]
              if(ind!=0):
                    f1 = ( emb_[ind].float() @ Wf1 ) + (h1 @ Uf1) + buf1
                    f1 = m(f1)
                    k1 = c1 * f1
                    g1 = (h1 @ Ug1) + (emb_[ind].float() @ Wg1) + bug1
                    g1 = tan_(g1)
                    i1 = (h1 @ Ui1) + (emb_[ind].float() @ Wi1) + bui1
                    i1 = m(i1)
                    j1 = g1 * i1
                    c1 = j1 + k1 
                    o1 = (h1 @ Uo1) + (emb_[ind].float() @ Wo1) + buo1
                    o1 = m(o1)
                    h1 = o1 * tan_(c1)  
        for i in range(0,len_): # maybe take average
              ind = X[len_-1-i]
              if(ind!=0):
                    f11 = ( emb_[ind].float() @ Wf11 ) + (h11 @ Uf11) + buf11
                    f11 = m(f11)
                    k11 = c11 * f11
                    g11 = (h11 @ Ug11) + (emb_[ind].float() @ Wg11) + bug11
                    g11 = tan_(g11)
                    i11 = (h11 @ Ui11) + (emb_[ind].float() @ Wi11) + bui11
                    i11 = m(i11)
                    j11 = g11 * i11
                    c11 = j11 + k11 
                    o11 = (h11 @ Uo11) + (emb_[ind].float() @ Wo11) + buo11
                    o11 = m(o11)
                    h11 = o11 * tan_(c11)
                    
        h3 = torch.cat((h1,h11),1)
        y2 = h3 @ V2 + bv2
        y2 = r_(y2)
        y = y2 @ V + bv
        
        y = s_(y)
        return y

In [ ]:
trainer = torch.optim.Adam(params, lr=0.001)

import math
for k in range(0,10000):
    if(k>8):
        trainer = torch.optim.Adam(params, lr=0.0001)
    print(k)
    for i in range(0,1000):
            print(i)
            j= random.randrange(1,39900,1)
            y_hat = models(X_train_2[j:j+1])
            l_ = cross_entropy(y_hat, y[j:j+1].long()) 
            
            #l_ = cross_entropy(y_hat.t(), y[j].long())            
            trainer.zero_grad()
            l_.backward()
            grad_clipping(params, 1.0)
            trainer.step()
            #print(emb_.grad[0:10])
    # Predicting on Training data
    '''
    preds = []
    for i in range(0,300):
       #print(i)
       y_hat = test(X_train_2[i])
       preds.append(torch.argmax(y_hat).cpu())
    from sklearn.metrics import accuracy_score
    print("training accuracy ", accuracy_score(y[0:300].cpu().numpy(), preds))
    '''
    preds = []
    for i in range(0,300):
       #print(i)
       y_hat = test(X_test_2[i])
       preds.append(torch.argmax(y_hat).cpu())
    from sklearn.metrics import accuracy_score
    print("training accuracy ", accuracy_score(y_[0:300].cpu().numpy(), preds))

0
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [23]:

def dropout_layer(X, dropout):
    if dropout == 1:
        return torch.zeros_like(X)
    if dropout == 0:
        return X
    mask = (torch.rand(X.shape) > dropout).float()
    return mask * X / (1.0 - dropout)

In [24]:
def grad_clipping(params_, theta=1.0):  #@save

    norm = torch.sqrt(sum(torch.sum((p.grad**2)) for p in params_))
    if norm > theta:
        for param in params_:
            param.grad[:] *= theta / norm

In [69]:
preds = []
for i in range(300,10000):
       #print(i)
       y_hat = test(X_test_2[i])
       preds.append(torch.argmax(y_hat).cpu())
from sklearn.metrics import accuracy_score
print("test accuracy ", accuracy_score(y_[300:10000].cpu().numpy(), preds))

test accuracy  0.8705154639175258


tensor([[ 0.0015, -0.0155, -0.0087,  ...,  0.0118,  0.0028, -0.0141],
        [-0.0159, -0.0146,  0.0082,  ..., -0.0008,  0.0154, -0.0024],
        [ 0.0130, -0.0072,  0.0055,  ..., -0.0019, -0.0052,  0.0174],
        ...,
        [-0.0091, -0.0009, -0.0015,  ..., -0.0052, -0.0004, -0.0010],
        [-0.0003,  0.0105, -0.0102,  ..., -0.0144,  0.0118, -0.0015],
        [-0.0048, -0.0109,  0.0015,  ...,  0.0289, -0.0033,  0.0144]],
       device='cuda:0', requires_grad=True)